### Importing libraries

In [3]:
#general libraries
import pandas as pd

#libraries for calculating geo distance
import geopandas as gpd
from shapely.geometry import Point

#libraries for importing data from URL
import requests

#libraries for allowing users to input information
import argparse

### Importing function for calculating geo distance

In [4]:
# transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
def to_mercator(lat, long):  
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

# return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
def distance_meters(start, finish):
    return start.distance(finish)

### Extracting raw data

#### Bicimad

Importing bicimad dataset from a csv:

In [5]:
bicimad_stations=pd.read_csv('../data/bicimad_stations.csv')

Taking only the intersting columns needed for the project and filtering the data:

In [6]:
bicimad_stations=bicimad_stations[["address","geometry_coordinates","name","dock_bikes"]]

Filtering out rows without geometry_coordinates:

In [7]:
bicimad_stations=bicimad_stations[bicimad_stations.geometry_coordinates.notnull()]

Checking if stations name are unique (they are not but the address it's different so this is not a problem):

In [8]:
bicimad_stations.groupby("name").filter(lambda x: len(x) > 1).sort_values("name")

,address,geometry_coordinates,name,dock_bikes
162,Calle Fernando el Católico nº 19,"[-3.708439, 40.4338516]",Fernando el Católico,18
201,Calle Fernando el Católico nº 61,"[-3.7149166, 40.4343611]",Fernando el Católico,13
89,Calle Goya nº 18,"[-3.6837876, 40.4250361]",General Pardiñas,11
188,Calle General Pardiñas nº 56,"[-3.6782777, 40.4290555]",General Pardiñas,5
121,Calle Méndez Álvaro nº 11,"[-3.6863218, 40.4013216]",Méndez Álvaro,15
182,Calle Retama nº 5,"[-3.6758888, 40.3941388]",Méndez Álvaro,8
128,Paseo de la Esperanza nº 2,"[-3.7064516, 40.4035988]",Paseo de la Esperanza,6
172,Paseo de la Esperanza nº 21,"[-3.7043611, 40.401]",Paseo de la Esperanza,16


Calculating coordinates:

In [9]:
bicimad_stations[["long_finish","lat_finish"]]=bicimad_stations['geometry_coordinates'].str.split(',', 1, expand=True)

In [10]:
bicimad_stations["lat_finish"]=bicimad_stations["lat_finish"].str.replace("]","", regex=True)

In [11]:
bicimad_stations["long_finish"]=bicimad_stations["long_finish"].str.replace("[","", regex=True)

Renaming field names:

In [12]:
bicimad_stations = bicimad_stations.rename(columns={'address': 'Station location', 'name': 'BiciMAD station'})

Deleting not needed columns:

In [13]:
bicimad_stations = bicimad_stations.drop(columns='geometry_coordinates')

In [14]:
bicimad_stations[["long_finish", "lat_finish"]] = bicimad_stations[["long_finish", 
                                                                   "lat_finish"]].apply(pd.to_numeric)

Calculating coordenades

In [15]:
%%time
coordinates_bicimad=bicimad_stations.apply(lambda x: to_mercator(x['lat_finish'],
                                                              x['long_finish']), axis=1)

CPU times: user 6.5 s, sys: 514 ms, total: 7.02 s
Wall time: 7.05 s


In [16]:
bicimad_stations["coordinates_bicimad"]=coordinates_bicimad

In [17]:
bicimad_stations.drop(['long_finish','lat_finish'], axis=1, inplace=True)

In [18]:
bicimad_stations

,Station location,BiciMAD station,dock_bikes,coordinates_bicimad
0,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol nº 1,Puerta del Sol B,22,POINT (4499234.670122574 -412347.498528782)
2,Calle Miguel Moya nº 1,Miguel Moya,12,POINT (4499599.340516399 -412820.3206106317)
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,4,POINT (4500679.707306497 -412940.3068148262)
4,Calle Manuela Malasaña nº 5,Malasaña,12,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...
259,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 1,0,POINT (4502177.655770458 -415179.5220676512)
260,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 2,12,POINT (4502140.920338497 -415172.8287427756)
261,Calle José Antonio Novais frente al nº 12,Facultad Biología,9,POINT (4502687.743941171 -415213.4906984274)
262,Avenida Complutense nº 23,Facultad Derecho,5,POINT (4502994.74083288 -415445.024322674)


#### Interesting Points

Defining public url for accesing to the datasets:

In [19]:
url_interest_points = 'https://datos.madrid.es/egob/catalogo/300356-0-monumentos-ciudad-madrid.json'

In [20]:
response_interest_points = requests.get(url_interest_points)

In [21]:
response_interest_points=response_interest_points.json()['@graph']

Converting json to pandas dataframe (renaming and adding columns)

In [22]:
interest_points = pd.json_normalize(response_interest_points)

In [23]:
interest_points=interest_points[["title","address.street-address","location.latitude","location.longitude"]]

In [24]:
interest_points = interest_points.rename(columns=
{'title': 'Place of interest', 'address.street-address': 'Place address',
 'location.latitude': 'lat_start','location.longitude': 'long_start'})

In [25]:
interest_points["Type of place"] = "Monumentos de la ciudad de Madrid" 

In [26]:
interest_points

,Place of interest,Place address,lat_start,long_start,Type of place
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,Monumentos de la ciudad de Madrid
1,A los Abuelos,C Alicún,40.479812,-3.661719,Monumentos de la ciudad de Madrid
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419,Monumentos de la ciudad de Madrid
3,Abogados Laboristas,C Atocha 55,40.412911,-3.700017,Monumentos de la ciudad de Madrid
4,Abstracta I,,40.421746,-3.662293,Monumentos de la ciudad de Madrid
...,...,...,...,...,...
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436127,-3.731997,Monumentos de la ciudad de Madrid
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436100,-3.732009,Monumentos de la ciudad de Madrid
1845,Yawar Fiesta,,NaN,NaN,Monumentos de la ciudad de Madrid
1846,'Sin título',,NaN,NaN,Monumentos de la ciudad de Madrid


Cleaning the dataframe:

In [27]:
interest_points=interest_points[interest_points.lat_start.notnull()]

In [28]:
interest_points=interest_points[interest_points.long_start.notnull()]

Cheking if points of interest are unique (names are not unique so minim level of detail is name+address):

In [29]:
pd.set_option('display.max_rows', 200)

In [30]:
interest_points.groupby("Place of interest").filter(lambda x: len(x) > 1).sort_values("Place of interest")

,Place of interest,Place address,lat_start,long_start,Type of place
12,"Agricultor, agricultura y progreso",Avda Complutense,40.441972,-3.729384,Monumentos de la ciudad de Madrid
13,"Agricultor, agricultura y progreso",C Principal de Provincias,40.412532,-3.739214,Monumentos de la ciudad de Madrid
14,"Agricultor, agricultura y progreso",PASEO CASTELLANA 101,40.453458,-3.691112,Monumentos de la ciudad de Madrid
37,Alfonso X El Sabio,PASEO RECOLETOS 20,40.423004,-3.690836,Monumentos de la ciudad de Madrid
38,Alfonso X El Sabio,JARDINES SABATINI,40.421540,-3.713312,Monumentos de la ciudad de Madrid
70,Antonio Bienvenida,C Alcalá 231,40.431575,-3.662272,Monumentos de la ciudad de Madrid
71,Antonio Bienvenida,C Alcalá 231,40.431561,-3.663014,Monumentos de la ciudad de Madrid
77,Antonio Machado,CALLE SAN ROMAN DEL VALLE 8,40.420280,-3.621731,Monumentos de la ciudad de Madrid
76,Antonio Machado,CALLE GENERAL ARRANDO 4,40.431625,-3.696805,Monumentos de la ciudad de Madrid
75,Antonio Machado,C Antonio Machado 55,40.468746,-3.719423,Monumentos de la ciudad de Madrid


Cheking if there are duplicates by name and address (6 duplicates):

In [31]:
interest_duplicates=interest_points.groupby(["Place of interest","Place address"]).filter(lambda x: len(x) > 1).sort_values("Place of interest")[["Place of interest","Place address"]]

In [32]:
interest_duplicates

,Place of interest,Place address
70,Antonio Bienvenida,C Alcalá 231
71,Antonio Bienvenida,C Alcalá 231
173,Casa de la Panadería,Pza Mayor 27
174,Casa de la Panadería,Pza Mayor 27
340,Enrique Tierno Galván,
343,Enrique Tierno Galván,
1356,Miguel de Cervantes,C Cervantes 2
1357,Miguel de Cervantes,C Cervantes 2
1686,Rey astur-leonés,C Méndez Núñez 1
1687,Rey astur-leonés,C Méndez Núñez 1


In [33]:
(interest_duplicates["Place of interest"]+interest_duplicates["Place address"]).unique()

array(['Antonio BienvenidaC Alcalá 231',
       'Casa de la PanaderíaPza Mayor 27', 'Enrique Tierno Galván ',
       'Miguel de CervantesC Cervantes 2',
       'Rey astur-leonésC Méndez Núñez 1',
       'XXV aniversario Colegio Mayor ArgentinoC Martín Fierro '],
      dtype=object)

Calculating coordinades:

In [34]:
interest_points[["long_start", "lat_start"]] = interest_points[["long_start", 
                                                                   "lat_start"]].apply(pd.to_numeric)

In [35]:
%%time
coordinates_interest_points=interest_points.apply(lambda x: to_mercator(x['lat_start'],
                                                              x['long_start']), axis=1)

CPU times: user 47.4 s, sys: 3.33 s, total: 50.7 s
Wall time: 51 s


In [36]:
interest_points["coordinates_interest_points"]=coordinates_interest_points

In [37]:
interest_points.drop(['long_start','lat_start'], axis=1, inplace=True)

In [38]:
interest_points

,Place of interest,Place address,Type of place,coordinates_interest_points
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...
1840,Wamba,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499284.447377896 -413608.0839695063)
1841,Wifredo. El Velloso,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499398.509382567 -413488.3182058548)
1842,X aniversario Colegio Mayor Argentino Nuestra ...,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501320.015285811 -415743.259706655)
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)


### Joining Data

Joining both tables with all posibilities for calculating the distance:

In [39]:
interest_points_bicimad = bicimad_stations.assign(key=0).merge(interest_points.assign(key=0), how='left', on = 'key')

In [40]:
interest_points_bicimad.drop('key', axis=1, inplace=True)

In [41]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,coordinates_bicimad,Place of interest,Place address,Type of place,coordinates_interest_points
0,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Puerta del Sol nº 1,Puerta del Sol A,15,POINT (4499223.648366921 -412373.2851572837),Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...,...,...,...,...
473083,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),Wamba,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499284.447377896 -413608.0839695063)
473084,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),Wifredo. El Velloso,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499398.509382567 -413488.3182058548)
473085,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),X aniversario Colegio Mayor Argentino Nuestra ...,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501320.015285811 -415743.259706655)
473086,Calle Zurbano nº 90,Zurbano,6,POINT (4501578.756909991 -411366.6317244973),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)


### Calculating distance

In [42]:
%%time
distance=interest_points_bicimad.apply(lambda x: distance_meters(x['coordinates_interest_points'],
                                                              x['coordinates_bicimad']), axis=1)

CPU times: user 7.66 s, sys: 113 ms, total: 7.77 s
Wall time: 7.88 s


In [43]:
interest_points_bicimad["distance"]=distance

In [44]:
interest_points_bicimad.drop(['coordinates_bicimad','coordinates_interest_points'], axis=1, inplace=True)

In [45]:
interest_points_bicimad=interest_points_bicimad.sort_values(["Place of interest","Station location"])

In [46]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,Place of interest,Place address,Type of place,distance
437250,Avenida Bruselas nº 56,Avenida Bruselas,5,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,6431.780052
469506,Avenida Complutense nº 23,Facultad Derecho,5,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,11536.258751
462338,Avenida Juan de Herrera frente a la calle Paul...,INEF,14,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,10735.369376
421122,Avenida Marqués de Corbera nº 12,Marqués de Corbera 12,8,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,4589.330269
422914,Avenida Marqués de Corbera nº 52,Marqués de Corbera 52,14,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,4939.620431
...,...,...,...,...,...,...,...
64163,Plaza del Cordón,Plaza del Cordón,18,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,3759.110762
1443,Puerta del Sol nº 1,Puerta del Sol A,15,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,2776.677372
3235,Puerta del Sol nº 1,Puerta del Sol B,22,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,2749.128671
92835,Ronda de Atocha nº 2,Ronda de Atocha,21,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,2962.047768


In [47]:
interest_points_bicimad[interest_points_bicimad.distance.notnull()==False]

,Station location,BiciMAD station,dock_bikes,Place of interest,Place address,Type of place,distance


### Filtering by the minimun distant:

In [48]:
nearest_BiciMAD_station=interest_points_bicimad[interest_points_bicimad['distance'] ==
                        interest_points_bicimad.groupby(['Place of interest','Place address'])['distance'].transform('min')]

In [49]:
nearest_BiciMAD_station

,Station location,BiciMAD station,dock_bikes,Place of interest,Place address,Type of place,distance
412162,Calle Arroyo del Olivar nº 6,Concordia,6,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,2012.572727
415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,1,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
448001,Paseo de la Castellana nº 298,Manuel Caldeiro,16,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,2633.107767
164883,Calle Goya nº 1,Plaza de Colón,4,AICO,Pza Colón,Monumentos de la ciudad de Madrid,110.126957
73475,Calle Atocha nº 54,Antón Martín,13,Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,127.684089
...,...,...,...,...,...,...,...
62769,Plaza del Cordón,Plaza del Cordón,18,Álvaro de Bazán,Pza Villa,Monumentos de la ciudad de Madrid,102.328354
451649,Calle Santa Hortensia nº 31,Corazón de María,6,Ángeles de la Paz,Avda Bruselas,Monumentos de la ciudad de Madrid,326.569317
415829,Calle Carlota O'Neill nº 32,Carlota O’Neill,1,Árbol de la Vida,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
65596,Plaza de Ramales,Plaza Ramales,8,Íñigo Arista,Pza Oriente,Monumentos de la ciudad de Madrid,108.278072


In [50]:
nearest_BiciMAD_station[(nearest_BiciMAD_station["Place of interest"]=="A las víctimas del Holocausto") | (nearest_BiciMAD_station["Place of interest"]=="A los Abuelos")]

,Station location,BiciMAD station,dock_bikes,Place of interest,Place address,Type of place,distance
415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,1,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
448001,Paseo de la Castellana nº 298,Manuel Caldeiro,16,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,2633.107767


### Saving a csv with the results:

In [51]:
nearest_BiciMAD_station.to_csv('../data/nearest_BiciMAD_station.csv')